In [1]:
import os
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings('ignore')

* trains_per_hour2 contains the average number of trains per hour of day (0-23) per direction at each of the stations

In [2]:
trains_per_hour2 = pd.read_excel("../../Data/avgTrainThroughput_byHour.xlsx")
trains_per_hour2.head()

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,HOURINT,AVG_TRAINS
0,MSTN_001,F06,1,Saturday,0,4
1,MSTN_001,F06,2,Saturday,0,3
2,MSTN_001,F06,1,Saturday,1,1
3,MSTN_001,F06,2,Saturday,1,2
4,MSTN_001,F06,2,Saturday,2,1


* Filter the trains per hour for only weekday information
* Categorize each of the time periods into peak, off peak and late night periods
* Keep only the station ID and average trains columns
* Group by the station IDs and average the AVG_TRAINS column
* Export

In [3]:
trains_per_hour2 = trains_per_hour2[trains_per_hour2['SERVICETYPE']=='Weekday']

In [4]:
def determine_period(row):
    if  5 <= row['HOURINT'] <= 10:
        return 'Peak'
    elif 15 <= row['HOURINT'] <= 19:
        return 'Peak'
    elif 11 <= row['HOURINT'] <= 14:
        return 'Off Peak'
    elif 20 <= row['HOURINT'] <= 22:
        return 'Off Peak'
    elif 23 == row['HOURINT']:
        return 'Late Night'
    elif 0 <= row['HOURINT'] <= 4:
        return 'Late Night'
    else:
        return 'Other'

trains_per_hour2['period'] = trains_per_hour2.apply(determine_period, axis=1)


In [5]:
trains_per_hour2.head()

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,HOURINT,AVG_TRAINS,period
81,MSTN_001,F06,1,Weekday,0,3,Late Night
82,MSTN_001,F06,2,Weekday,0,2,Late Night
83,MSTN_001,F06,1,Weekday,1,1,Late Night
84,MSTN_001,F06,2,Weekday,1,2,Late Night
85,MSTN_001,F06,1,Weekday,3,1,Late Night


In [6]:
os.makedirs('output', exist_ok=True)

In [7]:
trains_per_hour2 = trains_per_hour2[['STATION_ID', 'AVG_TRAINS']]
final_trains_per_hour = trains_per_hour2.groupby(['STATION_ID']).mean().reset_index()
final_trains_per_hour.to_csv("output/trains_per_hour_all_periods.csv")